**Q/A ANSWERING WITH LLAMA2**

**Summary**

LLAMA2 is a LLM model which was released by meta.Llama 2 is a collection of pretrained and fine-tuned generative text models ranging in scale from 7 billion to 70 billion parameters.This is the collab file for the 7B pretrained model.

Model Architecture Llama 2 is an auto-regressive language model that uses an optimized transformer architecture. The tuned versions use **supervised fine-tuning (SFT)** and **reinforcement learning with human feedback (RLHF)** to align to human preferences for helpfulness and safety.

**Intended Use Cases** Llama 2 is intended for commercial and research use in English. Tuned models are intended for assistant-like chat, whereas pretrained models can be adapted for a variety of natural language generation tasks.

This collab is created to show the basic implementation of LLAMA2 model on a intented pdf to have Q/A interaction. The model will interact with the pdf and it will either retrieve or generate a answer depending on the findings it finds in the pdf.


---


***NOTE***: To run this collab file, You have to load the collab file in

**GPU type** : T4

**Runtime shape** : High-RAM

To make this changes click on  Edit-> Notebook Settings-> and change to above settings.


---


The step by step guide to load the model and interact with it is provided below.



Step1: Installing all dependencies.

In [1]:
!pip -qqq install langchain apify-client faiss-cpu sentence_transformers accelerate bitsandbytes
!pip -q install langchain unstructured sentence_transformers faiss-cpu huggingface_hub OpenAI
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


**Step2**: Importing all libraries.Mostly libraries are from langchain because it makes LLM integration seamless easy.


In [2]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import ApifyDatasetLoader
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch

**Step3**:Loading embedding.

Here hugging face embedding is used as it is open source and free for commerical use.

Embedding is a technique in natural language processing (NLP) that represents words or phrases as vectors of real numbers.

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

**Step4**: Loading the dataset, here I have loaded a pdf called "football.pdf" but any other pdf or dataset can be loaded here.

In [4]:
!pip install pdfplumber
import pdfplumber

def pdf_loader(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        pages = pdf.pages

        documents = []
        for page in pages:
            text = page.extract_text()
            documents.append(Document(page_content=text))

    return documents

documents = pdf_loader("football.pdf")

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200
    )

docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 103.4 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


**Step5**: Loading LLAMA2 model.

It is most important and crucial step of building this model.

I have used BITSANDBYTESCONFIG to load model in 4bits. It is done to ensure the model doesn't take too much is CPU and GPU RAM.Although it might effect on the type of answer it generates, it's still a better model.

Then model is loaded using  AutoModelForCausalLM. I have used a sharded model because

*   Improved performance: Sharded
LLMs
can be loaded more quickly than monolithic LLMs, which can improve the performance of applications that use them.
*   Reduced memory usage: Sharded LLMs can be stored in smaller chunks of memory, which can reduce the memory requirements of applications that use them.





In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

**Step6**: Loading tokenizer.Each LLM provides it's own tokenizer.

Tokenizer is a tool that breaks text into smaller units, such as words, phrases, or tokens. This process is called tokenization

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side="left")

**Step7**: Creating a pipeline.A pipeline is a sequence of steps or operations that are executed in a specific order to achieve a particular goal.

This code creates a pipeline for text generation using the Hugging Face Transformers library. The pipeline takes a model, a tokenizer, and a number of parameters as input. The model is used to generate text, the tokenizer is used to tokenize the text, and the parameters control the generation process.

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=3000,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.2,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

**Step8**: Final step of this Q/A model is to retrieve the answer from our dataset/pdf. This is done by "RetrievalQA.from_chain_type".

Then we just have to pass our query and ask the model to process it. It will then retrieve or generate accordingly.

In [8]:
qa = RetrievalQA.from_chain_type(llm=local_llm,
                                 chain_type="stuff",
                                 retriever=db.as_retriever(k=2),
                                 return_source_documents=True,
                                 verbose=True)

query = "what is football?"
result = qa(query)
result['result']



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(



> Finished chain.


' It is a game where they kick a round object back and forth between them while running all over the field trying not to get tackled.\n\nAnswer: it is a game where they kick a round object back and forth between them while running all over the field trying not to get tackled.'

**Few other queries and their results.**

In [9]:
query = "Can a player be substituted in football during  match?"
result = qa(query)
result['result']



> Entering new RetrievalQA chain...


' Yes, but only once per half.\n\n\n'

**Challenges faced**

*   To load the Llama model as official version need persmission from meta.
*   Improve the model as it is a large model, I had to play with the parameters.

*   The time taken to load the embedding , model was more apart from that answer took too much time to generate.

**Improvements**

*    Time to generate has to be reduced.

*    Still a good amount of time has to be spent with parameters.

*    Improve model loading time and a good user interface to interact with it.

